# Database Setup

Data is originally from <br>
https://github.com/binlist<br>
https://github.com/iannuttall/binlist-data<br>
and manually entering bins. This data has been combined and modified.

Imports

In [ ]:
# Generic Import
import pandas as pd
from pymongo import MongoClient
import numpy as np
import json

db_cert = "ca-certificate.crt" #PATH/TO/ca-certificate.crt

Connect to/create MongoDB database

In [ ]:
def get_database(db_name):
    # Load connection string from creds.json
    with open('creds.json') as creds:
        CONNECTION_STRING = json.load(creds)['MongoDB']['connection_string'] + db_cert

    # Create a connection using MongoClient
    client = MongoClient(CONNECTION_STRING)

    return client[db_name] # return database

In [ ]:
db = get_database('bin_list')
bins = db['bins'] # create collection in db bin_list

# Enter Values manually into the databse

See all bins currently in database

In [ ]:
for bin in bins.find({}):
    #bins.delete_one(bin)
    print(bin['Bin'])

Let's add BINS manually

In [ ]:
response = input("Enter a BIN: ")
while response != "exit":
    if len(response) < 6:
        print("Please enter a 6 digit bin number.")
        response = input("Enter a BIN: ")
        continue
    if len([bin for bin in bins.find({'Bin': response})]) == 0:
        info = {'Bin': response}
        info['Type'] = input("Type of Card (Debit/Credit): ")
        info['Reloadable'] = input("Is the card Reloadable? (Y/N): ")
        info['Category'] = input("Category (Prepaid, Gift, Credit, Debit): ")
        info['Company'] = input("Company (Vanilla, MyVanilla, etc.): ")
        info['Distributor'] = input("Financial Distributor (InComm Financial Services): ")
        info['Issuer'] = input("Issuer (bank that issues): ")
        info['Website'] = input("Website (include https:// and trailing /): ")
        info['Customer Service'] = input('Support number (1-833-322-6760): ')
        info['Country'] = input("USA")
        print("Submitting: ")
        for k, v in info.items():
            print(f"{k}: {v}")
        bins.insert_one(info).inserted_id
        print("Saved")
        response = input("Enter a BIN: ")
    else:
        print("BIN Already Uploaded.")
        response = input("Enter a BIN: ")
print("Finished")

View number of bins in the database now

In [ ]:
len([bin for bin in bins.find({})])

# Adding CSV's to the database

Let's clean up the first CSV and then upload to the database

In [ ]:
bin1 = pd.read_csv('binlist-data.csv')
bin1

Let's drop unused columns from this CSV

In [ ]:
# bin1 = bin1.drop(columns = ['alpha_2', 'country', 'latitude', 'longitude'])
bin1

Create a for loop to add this information to the DB (we do NOT want to overwrite current data)

In [ ]:
def add_bin(bin_info):
    if len([bin for bin in bins.find({'Bin': bin_info['bin']})]) == 0:
        info = {'Bin': bin_info['bin']}
        info['Type'] = bin_info['type']
        info['Reloadable'] = ''
        info['Category'] = bin_info['type']
        info['Company'] = ''
        info['Distributor'] = ''
        info['Issuer'] = bin_info['issuer']
        info['Website'] = bin_info['bank_url']
        info['Customer Service'] = bin_info['bank_phone']
        info['Country'] = bin_info['alpha_3']
        bins.insert_one(info).inserted_id
    else:
        print("BIN Already Uploaded.")

In [ ]:
bin1.apply(add_bin, axis = 1)

Now we must redo the process for the second CSV

In [ ]:
bin2 = pd.read_csv('ranges.csv')
bin2

In [ ]:
bin2 = bin2.drop(columns = ['iin_end', 'number_length', 'number_luhn', 'scheme', 'bank_logo', 'bank_city'])
bin2.head(5)

In [ ]:
def add_bin2(bin_info):
    if len([bin for bin in bins.find({'Bin': bin_info['iin_start']})]) == 0:
        info = {'Bin': bin_info['iin_start']}
        info['Type'] = bin_info['brand']
        info['Reloadable'] = ''
        info['Category'] = bin_info['prepaid']
        info['Company'] = ''
        info['Distributor'] = ''
        info['Issuer'] = bin_info['bank_name']
        info['Website'] = bin_info['bank_url']
        info['Customer Service'] = bin_info['bank_phone']
        info['Country'] = bin_info['country']
        bins.insert_one(info).inserted_id
    else:
        print("BIN Already Uploaded.")

In [ ]:
bin2.apply(add_bin2, axis = 1)

Change string bins to integers to be compatible with API

In [ ]:
db = get_database('bin_list')
bins = db['bins'] # create collection in db bin_list
for bin in bins.find({'Bin': {'$type': 'string'}}):
    bins.update_one({'_id': bin['_id']}, {'$set': {'Bin': int(bin['Bin'])}})

Change NaN to empty strings (I didn't thoroughly check data before uploading to database, would have been MUCH quicker to change this in the pandas dataframe before uploading.)

In [ ]:
db = get_database('bin_list')
bins = db['bins'] # create collection in db bin_list
for column in ['Type', 'Reloadable', 'Category', 'Company', 'Distributor', 'Issuer', 'Website', 'Customer Service', 'Country']:
    for bin in bins.find({column: {'$eq': np.nan}}):
        bins.update_one({'_id': bin['_id']}, {'$set': {column: ''}})